In [1]:
import Pkg
Pkg.add("TextAnalysis")
using TextAnalysis

    Updating registry at `C:\Users\ethan\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed Snowball_jll ──────── v2.2.0+0
   Installed StrTables ─────────── v1.0.1
   Installed ConcurrentUtilities ─ v2.2.0
   Installed OpenSSL ───────────── v1.4.1
   Installed TranscodingStreams ── v0.9.13
   Installed WordTokenizers ────── v0.5.6
   Installed DataDeps ──────────── v0.7.10
   Installed TextAnalysis ──────── v0.7.3
   Installed OpenSSL_jll ───────── v3.0.8+0
   Installed Snowball ──────────── v0.1.0
   Installed HTML_Entities ─────── v1.0.1
   Installed HTTP ──────────────── v1.9.0
   Installed Languages ─────────── v0.4.3
    Updating `C:\Users\ethan\Project.toml`
  [a2db99b7] + TextAnalysis v0.7.3
    Updating `C:\Users\ethan\Manifest.toml`
  [d1d4a3ce] + BitFlags v0.1.7
  [944b1d66] + CodecZlib v0.7.1
  [f0e56b4a] + ConcurrentUtilities v2.2.0
  [124859b0] + DataDeps v0.7.10
  [7693890a] + HT

In [32]:
function create_Corpus(descs)
    st = "crps = Corpus(["
    t = "StringDocument("
    b = "),"
    en = "])"
    
    for i in eachindex(descs)
        st = st * t * "descs[" * "$i" *"]" * b
    end
    st = st * en
    return eval(Meta.parse(st))
end

create_Corpus (generic function with 1 method)

In [39]:
descs = ["may","be","not"]  #must be named descs 
create_Corpus(descs) 
update_lexicon!(crps) #must be named crps
lexicon(crps)
m = DocumentTermMatrix(crps)

A 3 X 3 DocumentTermMatrix

In [44]:
using TextAnalysis: NaiveBayesClassifier, fit!, predict

In [47]:
model = NaiveBayesClassifier([:burn_risk,:strangling_risk,:toxicity_risk])

NaiveBayesClassifier{Symbol}(String[], [:burn_risk, :toxicity_risk, :strangling_risk], Matrix{Int64}(undef, 0, 3))

In [56]:
#one example of burn risk
fit!(model,
    "The recalled children's sleepwear fails to meet 
    the federal flammability standards for children's sleepwear, 
    posing a risk of burn injuries to children.",
    :burn_risk)
#one example of strangling risk
fit!(model,
    "A young child can become entrapped by unzipping and entering the blanket,
    posing a risk of death by asphyxiation.",
    :strangling_risk)
#one example of toxicity risk
fit!(model,
"The recalled over-the-counter products contain the regulated substance acetaminophen
    which must be in child resistant packaging as required by the Poison Prevention
    Packaging Act (PPPA). The packaging of the products is not child resistant,
    posing a risk of poisoning if the contents are swallowed by young children.",
    :toxicity_risk)

NaiveBayesClassifier{Symbol}(["flammability", "sleepwear", "a", "for", "federal", ",", "risk", "'", "s", "."  …  "must", "be", "resistant", "PPPA", "over-the-counter", "are", "products", "as", ")", "which"], [:burn_risk, :toxicity_risk, :strangling_risk], [3 1 1; 5 1 1; … ; 1 2 1; 1 2 1])

In [57]:
#test a different description of burn damage 
predict(model,"The children's robes fail to meet the federal flammability standards 
    for children's sleepwear, posing a risk of burn injuries to children.")

Dict{Symbol, Float64} with 3 entries:
  :burn_risk       => 1.0
  :strangling_risk => 2.02292e-10
  :toxicity_risk   => 1.39702e-11

In [58]:
#test a different example of strangling risk
predict(model,"Young children can become entrapped in the space between 
    the exterior landing (hoistway) door and the interior elevator car
    door or gate if there is a hazardous gap and suffer serious injuries 
    or death when the elevator is called to another floor.")

Dict{Symbol, Float64} with 3 entries:
  :burn_risk       => 0.0034916
  :strangling_risk => 0.777768
  :toxicity_risk   => 0.21874

In [59]:
#test a different example of toxicity risk
predict(model,"Surface paint on the desks and chairs contains
    levels of lead that exceed the federal lead paint ban, 
    posing a lead poisoning hazard. The desks and chairs also 
    fail to comply with the federal lead content ban. 
    Lead is toxic if ingested by young children and can 
    cause adverse health effects.")

Dict{Symbol, Float64} with 3 entries:
  :burn_risk       => 8.66389e-9
  :strangling_risk => 2.28338e-6
  :toxicity_risk   => 0.999998